   **Music Genre Classification**

In [1]:
# feature extractoring and preprocessing data
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
from PIL import Image
import librosa.display
import pathlib
import csv


# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Keras
import keras

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#downlaoding dataset
!wget https://www.dropbox.com/s/yqg3c9mdmq665fy/genres.zip
     

In [3]:
!unzip '/content/genres.zip'

Archive:  /content/genres.zip
   creating: genres/
   creating: genres/blues/
  inflating: genres/blues/blues.00000.au  
  inflating: genres/blues/blues.00001.au  
  inflating: genres/blues/blues.00002.au  
  inflating: genres/blues/blues.00003.au  
  inflating: genres/blues/blues.00004.au  
  inflating: genres/blues/blues.00005.au  
  inflating: genres/blues/blues.00006.au  
  inflating: genres/blues/blues.00007.au  
  inflating: genres/blues/blues.00008.au  
  inflating: genres/blues/blues.00009.au  
  inflating: genres/blues/blues.00010.au  
  inflating: genres/blues/blues.00011.au  
  inflating: genres/blues/blues.00012.au  
  inflating: genres/blues/blues.00013.au  
  inflating: genres/blues/blues.00014.au  
  inflating: genres/blues/blues.00015.au  
  inflating: genres/blues/blues.00016.au  
  inflating: genres/blues/blues.00017.au  
  inflating: genres/blues/blues.00018.au  
  inflating: genres/blues/blues.00019.au  
  inflating: genres/blues/blues.00020.au  
  inflating: genres

In [4]:
!rm '/content/genres.zip

/bin/bash: -c: line 0: unexpected EOF while looking for matching `''
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [5]:
#Extracting the Spectogram for every Audio
cmap = plt.get_cmap('inferno')

plt.figure(figsize=(10,10))
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    pathlib.Path(f'/content/output/{g}').mkdir(parents=True, exist_ok=True)     
    for filename in os.listdir(f'/content/genres/{g}'):
        #print (filename)
        #print ({g})
        songname = f'/content/genres/{g}/{filename}'
        #print (songname)
        y, sr = librosa.load(songname, mono=True, duration=5)
        plt.specgram(y, NFFT=2048, Fs=2, Fc=0, noverlap=128, cmap=cmap, sides='default', mode='default', scale='dB');
        plt.axis('off');
        plt.savefig(f'/content/output/{g}/{filename[:-3].replace(".", "")}.png')
        plt.clf()

<Figure size 720x720 with 0 Axes>

In [ ]:
genres = 'blues classical'.split()
for g in genres:
  for filename in os.listdir(f'/content/genres/{g}'):       
      songname = f'/content/genres/{g}/{filename}'
      X = librosa.stft(y) #converts to short term fourier transform
      Xdb = librosa.amplitude_to_db(abs(X))
      plt.figure(figsize=(14, 5))
      librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz') 
      plt.colorbar()

In [7]:
#Extracting features from Spectogram
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [9]:
file = open('data.csv', 'w', newline='') #data.csv in write mode
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = 'blues classical country disco hiphop jazz metal pop reggae rock'.split()
for g in genres:
    for filename in os.listdir(f'/content/genres/{g}'):
        songname = f'/content/genres/{g}/{filename}'
        y, sr = librosa.load(songname, mono=True, duration=30)
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        rmse = librosa.feature.rms(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='') #data.csv in append mode
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [10]:
data = pd.read_csv('data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,...,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,blues.00016.au,0.304322,0.081461,1455.001672,1825.885415,3009.856800,0.061351,-233.763596,120.673508,-3.171085,...,2.537329,-1.240455,8.439245,-0.210692,-3.906118,-4.478783,-0.012436,1.293613,-2.084796,blues
1,blues.00040.au,0.397725,0.134892,2272.495079,2356.855937,5108.319111,0.097657,-110.260086,90.701813,-2.527487,...,5.952405,-8.033586,4.239867,-16.485552,6.542503,-5.695944,-3.640863,-4.856884,1.854517,blues
2,blues.00020.au,0.302732,0.075387,1389.955510,1911.485152,3005.773491,0.052560,-230.412994,127.094185,7.151100,...,2.561243,-4.329453,6.817760,-6.157660,-6.214757,-4.515434,-1.850599,-0.539056,1.508026,blues
3,blues.00006.au,0.291308,0.141796,1459.078483,1388.913312,2795.616429,0.073028,-190.149460,130.296951,-36.344139,...,-3.125678,-6.593118,-9.942267,-6.537224,-10.064754,-10.912171,-6.972478,-3.449033,-6.495510,blues
4,blues.00009.au,0.274009,0.081352,1817.516386,1973.739070,3944.451148,0.079215,-213.180801,115.152794,-11.716267,...,3.784771,0.225668,-5.113637,-0.413064,-1.184537,-1.920715,-2.293728,2.891266,-4.233204,blues


In [11]:
data.shape

(1000, 28)

In [12]:
data = data.drop(['filename'],axis=1)

In [13]:
genre_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [14]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [16]:
len(y_train)

800

In [17]:
len(y_test)

200

In [18]:
X_train[10]

array([ 1.27227977,  1.24122185,  0.82974955,  0.15046842,  0.56149918,
        1.43721207,  1.44602418, -0.63356776, -1.00189897,  1.48853097,
       -1.33608898,  1.01999429, -1.62419369,  1.17031336, -0.7289337 ,
        1.27919527, -1.08021768,  2.20400368, -1.00297725,  1.78358017,
       -1.14386895,  1.42362843, -1.03998427,  0.69024394, -1.20006467,
        1.29852633])

In [19]:
#Network Building
from keras import models
from keras import layers
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

In [20]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [21]:
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)

Epoch 1/20
7/7 [==============================] - 1s 5ms/step - loss: 2.1907 - accuracy: 0.2425
Epoch 2/20
7/7 [==============================] - 0s 4ms/step - loss: 1.8525 - accuracy: 0.3487
Epoch 3/20
7/7 [==============================] - 0s 4ms/step - loss: 1.6360 - accuracy: 0.3812
Epoch 4/20
7/7 [==============================] - 0s 5ms/step - loss: 1.4745 - accuracy: 0.4688
Epoch 5/20
7/7 [==============================] - 0s 5ms/step - loss: 1.3381 - accuracy: 0.5275
Epoch 6/20
7/7 [==============================] - 0s 4ms/step - loss: 1.2199 - accuracy: 0.5775
Epoch 7/20
7/7 [==============================] - 0s 4ms/step - loss: 1.1287 - accuracy: 0.6200
Epoch 8/20
7/7 [==============================] - 0s 4ms/step - loss: 1.0475 - accuracy: 0.6600
Epoch 9/20
7/7 [==============================] - 0s 4ms/step - loss: 0.9956 - accuracy: 0.6687
Epoch 10/20
7/7 [==============================] - 0s 4ms/step - loss: 0.9282 - accuracy: 0.7088
Epoch 11/20
7/7 [======================

In [23]:
test_loss, test_acc = model.evaluate(X_test,y_test)

7/7 [==============================] - 0s 7ms/step - loss: 1.0035 - accuracy: 0.6900


In [24]:
print('test_acc: ',test_acc)

test_acc:  0.6899999976158142


In [25]:
#Validation
x_val = X_train[:200]
partial_x_train = X_train[200:]

y_val = y_train[:200]
partial_y_train = y_train[200:]

In [26]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(partial_x_train,
          partial_y_train,
          epochs=30,
          batch_size=512,
          validation_data=(x_val, y_val))
results = model.evaluate(X_test, y_test)

Epoch 1/30
2/2 [==============================] - 1s 205ms/step - loss: 2.3173 - accuracy: 0.0833 - val_loss: 2.1555 - val_accuracy: 0.3100
Epoch 2/30
2/2 [==============================] - 0s 39ms/step - loss: 2.1150 - accuracy: 0.3750 - val_loss: 2.0354 - val_accuracy: 0.3550
Epoch 3/30
2/2 [==============================] - 0s 42ms/step - loss: 1.9559 - accuracy: 0.3767 - val_loss: 1.9058 - val_accuracy: 0.3550
Epoch 4/30
2/2 [==============================] - 0s 40ms/step - loss: 1.7877 - accuracy: 0.3883 - val_loss: 1.7847 - val_accuracy: 0.3950
Epoch 5/30
2/2 [==============================] - 0s 44ms/step - loss: 1.6385 - accuracy: 0.4133 - val_loss: 1.6932 - val_accuracy: 0.4100
Epoch 6/30
2/2 [==============================] - 0s 42ms/step - loss: 1.5196 - accuracy: 0.4400 - val_loss: 1.6236 - val_accuracy: 0.4000
Epoch 7/30
2/2 [==============================] - 0s 44ms/step - loss: 1.4246 - accuracy: 0.4817 - val_loss: 1.5377 - val_accuracy: 0.4550
Epoch 8/30
2/2 [==========

In [27]:
results

[1.0995689630508423, 0.6299999952316284]

In [28]:
predictions = model.predict(X_test)

7/7 [==============================] - 0s 3ms/step


In [45]:
predictions[0].shape

(10,)

In [46]:
np.sum(predictions[0])

0.99999994

In [47]:
np.argmax(predictions[0])

8

In [56]:
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

7/7 [==============================] - 0s 8ms/step


In [57]:
accuracy_score(y_test, classes_x)
     

0.63